In [2]:
## Relevant imports

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import re

from collections import defaultdict

# Tokenizer imports
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.tokenize import WordPunctTokenizer
from nltk.tokenize import regexp_tokenize

# NLTK corpus and stemming/lemmatizer imports
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

# Scikit-learn packages
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm, linear_model
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Gensim imports
import gensim

In [3]:
data = pd.read_csv("/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv")

In [4]:
data.shape

(50000, 2)

In [5]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [6]:
data.sentiment.unique()

array(['positive', 'negative'], dtype=object)

In [7]:
data.sentiment.value_counts()

positive    25000
negative    25000
Name: sentiment, dtype: int64

In [8]:
data.dtypes

review       object
sentiment    object
dtype: object

In [9]:
data.review.duplicated().sum()

418

In [10]:
data.drop_duplicates(keep = "first", inplace = True)
data.shape

(49582, 2)

In [11]:
data.isna().sum()

review       0
sentiment    0
dtype: int64

In [12]:
# Convert reviews to lowercase

data.review = data.review.apply(lambda x: str(x).lower())

In [13]:
data.reset_index(inplace = True)

In [14]:
data = data.drop("index", axis = 1)

In [15]:
data

,review,sentiment
0,one of the other reviewers has mentioned that ...,positive
1,a wonderful little production. <br /><br />the...,positive
2,i thought this was a wonderful way to spend ti...,positive
3,basically there's a family where a little boy ...,negative
4,"petter mattei's ""love in the time of money"" is...",positive
...,...,...
49577,i thought this movie did a down right good job...,positive
49578,"bad plot, bad dialogue, bad acting, idiotic di...",negative
49579,i am a catholic taught in parochial elementary...,negative
49580,i'm going to have to disagree with the previou...,negative


In [16]:
def strip_html(raw_text):
  find_html = re.compile('<.*?>')
  clean_text = re.sub(find_html, '', raw_text)
  return clean_text

In [17]:
data.review = data.review.apply(lambda x: strip_html(x))

In [18]:
data

,review,sentiment
0,one of the other reviewers has mentioned that ...,positive
1,a wonderful little production. the filming tec...,positive
2,i thought this was a wonderful way to spend ti...,positive
3,basically there's a family where a little boy ...,negative
4,"petter mattei's ""love in the time of money"" is...",positive
...,...,...
49577,i thought this movie did a down right good job...,positive
49578,"bad plot, bad dialogue, bad acting, idiotic di...",negative
49579,i am a catholic taught in parochial elementary...,negative
49580,i'm going to have to disagree with the previou...,negative


In [19]:
# Running WhiteSpace tokenizer 
wpTokenizer = WordPunctTokenizer()
data["review_tokenized"] = [wpTokenizer.tokenize(text) for text in data["review"]]

In [20]:
data

,review,sentiment,review_tokenized
0,one of the other reviewers has mentioned that ...,positive,"[one, of, the, other, reviewers, has, mentione..."
1,a wonderful little production. the filming tec...,positive,"[a, wonderful, little, production, ., the, fil..."
2,i thought this was a wonderful way to spend ti...,positive,"[i, thought, this, was, a, wonderful, way, to,..."
3,basically there's a family where a little boy ...,negative,"[basically, there, ', s, a, family, where, a, ..."
4,"petter mattei's ""love in the time of money"" is...",positive,"[petter, mattei, ', s, "", love, in, the, time,..."
...,...,...,...
49577,i thought this movie did a down right good job...,positive,"[i, thought, this, movie, did, a, down, right,..."
49578,"bad plot, bad dialogue, bad acting, idiotic di...",negative,"[bad, plot, ,, bad, dialogue, ,, bad, acting, ..."
49579,i am a catholic taught in parochial elementary...,negative,"[i, am, a, catholic, taught, in, parochial, el..."
49580,i'm going to have to disagree with the previou...,negative,"[i, ', m, going, to, have, to, disagree, with,..."


In [21]:
# Stopwords removal & WordNet lemmatization 

# Define POS tags 
tag_map = defaultdict(lambda : wordnet.NOUN)
tag_map['J'] = wordnet.ADJ
tag_map['V'] = wordnet.VERB
tag_map['R'] = wordnet.ADV

In [22]:
for index, text in enumerate(data.review_tokenized):
    if index % 100 == 0:
        print(index)
#     print("-" * 50)
    word_list = []
    wordnet_lemmatizer = WordNetLemmatizer()
    for word, tag in pos_tag(text):
        if word not in stopwords.words("english") and word.isalpha():
            word_processed = wordnet_lemmatizer.lemmatize(word, tag_map[tag[0]])
            word_list.append(word_processed)
    data.loc[index, "review_tokenized_cleaned"] = str(word_list)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [23]:
data

,review,sentiment,review_tokenized,review_tokenized_cleaned
0,one of the other reviewers has mentioned that ...,positive,"[one, of, the, other, reviewers, has, mentione...","['one', 'reviewer', 'mention', 'watch', 'oz', ..."
1,a wonderful little production. the filming tec...,positive,"[a, wonderful, little, production, ., the, fil...","['wonderful', 'little', 'production', 'filming..."
2,i thought this was a wonderful way to spend ti...,positive,"[i, thought, this, was, a, wonderful, way, to,...","['think', 'wonderful', 'way', 'spend', 'time',..."
3,basically there's a family where a little boy ...,negative,"[basically, there, ', s, a, family, where, a, ...","['basically', 'family', 'little', 'boy', 'jake..."
4,"petter mattei's ""love in the time of money"" is...",positive,"[petter, mattei, ', s, "", love, in, the, time,...","['petter', 'mattei', 'love', 'time', 'money', ..."
...,...,...,...,...
49577,i thought this movie did a down right good job...,positive,"[i, thought, this, movie, did, a, down, right,...","['think', 'movie', 'right', 'good', 'job', 'cr..."
49578,"bad plot, bad dialogue, bad acting, idiotic di...",negative,"[bad, plot, ,, bad, dialogue, ,, bad, acting, ...","['bad', 'plot', 'bad', 'dialogue', 'bad', 'act..."
49579,i am a catholic taught in parochial elementary...,negative,"[i, am, a, catholic, taught, in, parochial, el...","['catholic', 'taught', 'parochial', 'elementar..."
49580,i'm going to have to disagree with the previou...,negative,"[i, ', m, going, to, have, to, disagree, with,...","['go', 'disagree', 'previous', 'comment', 'sid..."


In [24]:
data.review_tokenized_cleaned.isna().sum()

0

In [25]:
train_X, test_X, train_y, test_y = model_selection.train_test_split(data.review_tokenized_cleaned, data.sentiment, test_size = 0.3, random_state =1)

In [26]:
print(train_X.shape)
print(test_X.shape)
print(train_y.shape)
print(test_y.shape)

(34707,)
(14875,)
(34707,)
(14875,)


In [27]:
test_y.value_counts()

negative    7461
positive    7414
Name: sentiment, dtype: int64

In [28]:
train_y.value_counts()

positive    17470
negative    17237
Name: sentiment, dtype: int64

In [29]:
label_enc = LabelEncoder()
train_y = label_enc.fit_transform(train_y)
test_y = label_enc.transform(test_y)

In [30]:
print(np.unique(test_y, return_counts = True))
print(np.unique(train_y, return_counts = True))

(array([0, 1]), array([7461, 7414]))
(array([0, 1]), array([17237, 17470]))


In [31]:
tfidf_vect = TfidfVectorizer(max_features = 5000)
tfidf_vect.fit(data.review_tokenized_cleaned)

TfidfVectorizer(max_features=5000)

In [32]:
train_X_tfidf = tfidf_vect.transform(train_X)
test_X_tfidf = tfidf_vect.transform(test_X)

### Modelling Multinomial Naives Bayes

In [33]:
train_X_tfidf_dense = train_X_tfidf.todense()
test_X_tfidf_dense = test_X_tfidf.todense()

In [34]:
nb_model = naive_bayes.GaussianNB()
nb_model.fit(train_X_tfidf_dense, train_y)

GaussianNB()

In [35]:
preds_nb = nb_model.predict(test_X_tfidf_dense)

In [36]:
preds_nb.shape

(14875,)

In [37]:
accuracy_score(preds_nb, test_y)

0.7878991596638656

In [38]:
confusion_matrix(test_y, preds_nb)

array([[5938, 1523],
       [1632, 5782]])

In [39]:
print(classification_report(test_y, preds_nb))

              precision    recall  f1-score   support

           0       0.78      0.80      0.79      7461
           1       0.79      0.78      0.79      7414

    accuracy                           0.79     14875
   macro avg       0.79      0.79      0.79     14875
weighted avg       0.79      0.79      0.79     14875



### Support Vector Machine Classifier

Training can take some time, grab a coffee in the meanwhile :)

In [40]:
svm = svm.SVC(C = 1.0, kernel = "linear", degree = 3, gamma = "auto")
svm.fit(train_X_tfidf, train_y)

SVC(gamma='auto', kernel='linear')

In [41]:
preds_svm = svm.predict(test_X_tfidf)
print(preds_svm.shape)

(14875,)


In [42]:
accuracy_score(preds_svm, test_y)

0.8836302521008403

In [43]:
print(classification_report(test_y, preds_svm))

              precision    recall  f1-score   support

           0       0.90      0.87      0.88      7461
           1       0.87      0.90      0.89      7414

    accuracy                           0.88     14875
   macro avg       0.88      0.88      0.88     14875
weighted avg       0.88      0.88      0.88     14875



### Logistic Regression

In [44]:
log_reg = linear_model.LogisticRegression(solver = "lbfgs")
log_reg.fit(train_X_tfidf, train_y)

LogisticRegression()

In [45]:
preds_log_reg = log_reg.predict(test_X_tfidf)
preds_log_reg.shape

(14875,)

In [46]:
accuracy_score(preds_log_reg, test_y)

0.8863193277310925

In [47]:
print(classification_report(test_y, preds_log_reg))

              precision    recall  f1-score   support

           0       0.90      0.87      0.88      7461
           1       0.87      0.90      0.89      7414

    accuracy                           0.89     14875
   macro avg       0.89      0.89      0.89     14875
weighted avg       0.89      0.89      0.89     14875

